Fitur yang dipakai :
- TFIDF Word
- WOrd Shape
- WH Word

In [1]:
import pandas as pd
import numpy as np
import csv
from nltk.tokenize import word_tokenize
import pickle
from sklearn import preprocessing
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
import re 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def wh_tag(doc):
    wh_tag_list = {'apa': 1, 'apakah': 2, 'dimana': 3, 'di mana': 3, 'bagaimana':4, 'bagaimanakah':4, 'mengapa':5, 'kenapa':6, 'siapa':7, 'siapakah':7, 'kapan':8, 'kemana':9, 'dari mana':10, 'berapa':11, 'berapakah':11, 'mana':12, 'manakah':12, 'dimanakah':13}
    result = []
    wh_tags = []
    for t in word_tokenize(doc):
        if t in wh_tag_list:
            wh_tags.append(wh_tag_list[t])
    if not wh_tags:#If empty
        result.append(0) #none tag
    else:
        result.append(wh_tags[0])
    return result

In [38]:
sen_class = 1

target = 'dataset-question-classification-csv.csv'
with open(target,'r',encoding='utf-8') as csvFile:
    csvReader = csv.reader(csvFile, delimiter=';')
    next(csvReader)
    
    text = []
    classes = []
    tag_wh_new = []
    w_shape_upper = []
    w_shape_lower = []
    w_shape_digit = []
    w_shape_mix = []
    w_shape_other = []
    
    for row in csvReader:
        sentence = row[0].lower()
        
        if sen_class == 1:
            sentenceclass =row[1] #row[1]=coarse - row[2]=fine
        elif sen_class == 2:
            sentenceclass =row[2] #row[1]=coarse - row[2]=fine
        classes.append(sentenceclass)
        
        #Word Shape
        shape_upper = 0
        shape_lower = 0
        shape_digit = 0
        shape_mix = 0
        shape_other = 0
        words = word_tokenize(sentence)
        islower = len([word for word in words if word.islower()])
        shape_lower += islower
        isupper = len([word for word in words if word.isupper()])
        shape_upper += isupper
        isdigit = len([word for word in words if word.isdigit()])
        shape_digit += isdigit
        a = [word for word in words if not word.islower() and not word.isupper() and not word.isdigit()]
        for x in a:
            if x.isalpha() == True:
                shape_mix += 1
            if x.isalpha() == False:
                shape_other+=1
        w_shape_upper.append(shape_upper)
        w_shape_lower.append(shape_lower)
        w_shape_digit.append(shape_digit)
        w_shape_mix.append(shape_mix)
        w_shape_other.append(shape_other)
                
        #WH Word
        tag_wh_new.append(wh_tag(sentence.lower()))
        
        text.append(sentence)

In [39]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_feature = tfidf_vectorizer.fit_transform(text)
documents_text = tfidf_feature.toarray()
documents_text = documents_text.tolist()



# Insert Word Shape
i=0
for xxxxx in documents_text:
    documents_text[i].insert(0,w_shape_upper[i])
    i+=1
i=0
for xxxxx in documents_text:
    documents_text[i].insert(0,w_shape_lower[i])
    i+=1
i=0
for xxxxx in documents_text:
    documents_text[i].insert(0,w_shape_digit[i])
    i+=1
i=0
for xxxxx in documents_text:
    documents_text[i].insert(0,w_shape_mix[i])
    i+=1
i=0
for xxxxx in documents_text:
    documents_text[i].insert(0,w_shape_other[i])
    i+=1

# Insert WH Tag 
i=0
for xxxxx in documents_text:
    documents_text[i].insert(0,tag_wh_new[i][0])
    i+=1
    
    
    
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

a_train, a_test, b_train, b_test = train_test_split(documents_text, classes, test_size=0.30, random_state=42)

In [40]:
#SVM model Coarse Category
from sklearn import svm
model = svm.SVC(kernel='linear')
model.fit(a_train,b_train)

predictions = model.predict(a_test)
print(classification_report(b_test,predictions,digits=3))
print('\n\n')

              precision    recall  f1-score   support

        ABBR      0.947     0.857     0.900        21
        DESC      0.857     0.750     0.800        48
        ENTY      0.829     0.934     0.878       166
         HUM      0.978     0.891     0.933       101
         LOC      0.869     0.803     0.835        66
         NUM      0.979     0.986     0.983       143

    accuracy                          0.905       545
   macro avg      0.910     0.870     0.888       545
weighted avg      0.908     0.905     0.904       545






In [37]:
#SVM model Fine Category
from sklearn import svm
model = svm.SVC(kernel='linear')
model.fit(a_train,b_train)

predictions = model.predict(a_test)
print(classification_report(b_test,predictions,digits=3))
print('\n\n')

                precision    recall  f1-score   support

  abbreviation      0.500     1.000     0.667         5
        animal      0.800     0.857     0.828        14
          body      1.000     0.667     0.800         6
          city      1.000     0.750     0.857         8
          code      1.000     0.750     0.857         4
         color      1.000     0.900     0.947        10
         count      0.633     0.886     0.738        35
       country      0.833     0.833     0.833        12
      creation      0.500     0.786     0.611        14
      currency      1.000     0.800     0.889         5
          date      0.929     0.867     0.897        15
    definition      0.350     1.000     0.519         7
   description      0.429     0.375     0.400         8
        dismed      1.000     0.250     0.400        12
      distance      0.917     0.917     0.917        12
     entyother      0.400     0.533     0.457        15
         event      0.375     0.750     0.500  

c:\users\fandy\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
